In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=3d80def369253c79222dc4e588e5069535a69b0cf4435713def14e1ed7ce884f
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 5.0MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F7 '''
def F7(X):
    f = bn.F7()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.5995249632245891 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_7_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_7_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=886324, Mon Apr 13 18:39:40 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 5.183404662865697e+03 1.0e+00 2.39e+00  2e+00  2e+00 0:00.0
    2     30 2.361110954397513e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.0
    3     45 2.421275475493161e+03 1.1e+00 2.33e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5183.40466287] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8863.39390909] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -8.589294029523877e+03 1.4e+00 3.20e-01  3e-01  3e-01 0:01.8
  200   3000 -9.063266024448818e+03 1.5e+00 2.02e-02  2e-02  2e-02 0:03.5
  300   4500 -9.065030530780510e+03 1.5e+00 1.19e-03  1e-03  1e-03 0:05.3
  400   6000 -9.065035668544424e+03 1.6e+00 9.64e-05  8e-05  9e-05 0:07.0
  500   7500 -9.065035686620478e+03 1.6e+00 7.03e-06  6e-06  6e-06 0:08.8
  600   9000 -9.065035686716357e+03 1.7e+00 3.78e-07  3e-07  3e-07 0:10.4
  700  10500 -9.065035686716645e+03 1.8e+00 4.26e-08  3e-08  4e-08 0:12.2
  800  12000 -9.065035686716645e+03 1.9e+00 2.69e-08  2e-08  2e-08 0:14.0
  900  13500 -9.065035686716648e+03 2.1e+00 2.49e-08  2e-08  2e-08 0:15.7
 1000  15000 -9.065035686716645e+03 2.2e+00 2.31e-08  2e-08  2e-08 0:17.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -9.065035686716648e+03 2.3e+00 1.95e-08  1e-08  2e-08 0:19.1
 1200  18000 -9.065035686716645e+03 2.5e+00 1.48e-08  1e-08  1e-08 0:20.8
 1300  19500 -9.065035686716648e+03 2.7e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8403.36693887] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=8403.366939, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=8403.366939, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -4.718568701797718e+03 1.6e+00 8.12e-01  7e-01  9e-01 0:02.2
  200   3000 -6.406668781754268e+03 1.9e+00 2.48e-01  2e-01  3e-01 0:04.4
  300   4500 -6.732093903916084e+03 2.4e+00 7.79e-02  6e-02  1e-01 0:06.6
  400   6000 -6.752514541119770e+03 2.9e+00 3.45e-02  2e-02  5e-02 0:08.8
  500   7500 -6.760039882901454e+03 4.3e+00 1.52e-02  1e-02  2e-02 0:10.9
  600   9000 -6.760922416715450e+03 5.4e+00 6.55e-03  4e-03  1e-02 0:13.1
  700  10500 -6.761427920017799e+03 8.7e+00 5.71e-03  3e-03  1e-02 0:15.3
  800  12000 -6.761528144248819e+03 9.6e+00 1.62e-03  8e-04  4e-03 0:17.4
  900  13500 -6.761534552436318e+03 1.0e+01 3.63e-04  2e-04  1e-03 0:19.6
 1000  15000 -6.761534858307356e+03 1.1e+01 5.40e-05  2e-05  1e-04 0:21.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -6.761534863313243e+03 1.1e+01 7.92e-06  3e-06  2e-05 0:24.0
 1200  18000 -6.761534863398781e+03 1.2e+01 8.17e-07  3e-07  2e-06 0:26.2
 1300  19500 -6.761534863399502e+03 1.2e

...

  100   1500 -6.094962882068821e+03 1.5e+00 4.65e-01  4e-01  5e-01 0:02.2
  200   3000 -6.763245765683985e+03 1.6e+00 8.25e-02  7e-02  9e-02 0:04.3
  300   4500 -6.781282442540838e+03 1.9e+00 1.57e-02  1e-02  2e-02 0:06.5
  400   6000 -6.783239751445086e+03 3.4e+00 1.10e-02  8e-03  2e-02 0:08.6
  500   7500 -6.784336580541614e+03 6.5e+00 7.49e-03  5e-03  2e-02 0:10.8
  600   9000 -6.784495884595639e+03 7.4e+00 2.47e-03  1e-03  6e-03 0:12.9
  700  10500 -6.784518481554182e+03 7.7e+00 1.04e-03  6e-04  2e-03 0:15.1
  800  12000 -6.784520625862151e+03 8.5e+00 2.13e-04  1e-04  5e-04 0:17.3
  900  13500 -6.784520777052401e+03 8.9e+00 7.97e-05  4e-05  2e-04 0:19.4
 1000  15000 -6.784520785457755e+03 9.0e+00 9.88e-06  5e-06  3e-05 0:21.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -6.784520785678709e+03 9.4e+00 1.90e-06  8e-07  6e-06 0:23.6
 1200  18000 -6.784520785682551e+03 9.5e+00 1.82e-07  7e-08  5e-07 0:25.7
 1300  19500 -6.784520785682596e+03 9.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [11373.35420141] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9945.54976428] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -8.409258964692886e+03 1.5e+00 4.04e-01  4e-01  4e-01 0:02.1
  200   3000 -9.063032672200068e+03 1.6e+00 2.55e-02  2e-02  3e-02 0:04.1
  300   4500 -9.065027293240186e+03 1.6e+00 1.65e-03  1e-03  2e-03 0:06.1
  400   6000 -9.065035661622504e+03 1.6e+00 1.00e-04  8e-05  9e-05 0:08.0
  500   7500 -9.065035686629537e+03 1.6e+00 6.56e-06  5e-06  6e-06 0:10.1
  600   9000 -9.065035686716081e+03 1.7e+00 4.50e-07  3e-07  4e-07 0:12.1
  700  10500 -9.065035686716645e+03 1.7e+00 4.85e-08  4e-08  4e-08 0:14.1
  800  12000 -9.065035686716648e+03 1.9e+00 2.87e-08  2e-08  2e-08 0:16.1
  900  13500 -9.065035686716648e+03 2.2e+00 2.47e-08  2e-08  2e-08 0:18.1
 1000  15000 -9.065035686716648e+03 2.3e+00 2.00e-08  1e-08  2e-08 0:20.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -9.065035686716648e+03 2.5e+00 2.56e-08  2e-08  2e-08 0:22.1
 1200  18000 -9.065035686716645e+03 2.8e+00 2.19e-08  1e-08  2e-08 0:24.1
 1300  19500 -9.065035686716645e+03 3.0e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8403.36693887] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=8403.366939, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=8403.366939, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -5.840850337957694e+03 1.5e+00 6.27e-01  6e-01  7e-01 0:02.1
  200   3000 -6.671230150723506e+03 1.6e+00 1.02e-01  8e-02  1e-01 0:04.3
  300   4500 -6.723222620601873e+03 2.7e+00 5.30e-02  4e-02  7e-02 0:06.4
  400   6000 -6.752684848910987e+03 4.3e+00 4.28e-02  3e-02  7e-02 0:08.5
  500   7500 -6.760649371851154e+03 5.7e+00 1.39e-02  8e-03  3e-02 0:10.7
  600   9000 -6.761414217463798e+03 7.6e+00 5.53e-03  3e-03  1e-02 0:12.9
  700  10500 -6.761532710966936e+03 9.1e+00 1.28e-03  7e-04  3e-03 0:14.9
  800  12000 -6.761534822780195e+03 9.5e+00 1.70e-04  8e-05  4e-04 0:17.1
  900  13500 -6.761534862804745e+03 9.8e+00 2.17e-05  1e-05  5e-05 0:19.3
 1000  15000 -6.761534863393705e+03 1.0e+01 2.31e-06  1e-06  5e-06 0:21.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -6.761534863399451e+03 1.1e+01 2.51e-07  1e-07  6e-07 0:23.6
 1200  18000 -6.761534863399505e+03 1.1e+01 5.60e-08  2e-08  1e-07 0:25.8
 1300  19500 -6.761534863399515e+03 1.1e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F7_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F7(X_Values.iloc[i,:50])
    SVM_Fun [i] = F7(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F7(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([598.89274641]), count=array([30])) 1.1368683772161603e-13
SVM
ModeResult(mode=array([2295.76456079]), count=array([1])) 3064.5041230381894
ELN
ModeResult(mode=array([3972.66066106]), count=array([11])) 153.68622058966952
